In [1]:
from collections import defaultdict
from itertools import pairwise

import polars as pl
from ortools.sat.python import cp_model
from solver import solve

armor = pl.read_parquet("data/armor_pieces.parquet")
charms = pl.read_parquet("data/charms.parquet")
jewels = pl.read_parquet("data/jewels.parquet")
talents = pl.read_parquet("data/talents.parquet")
weapons = pl.read_parquet("data/weapons.parquet")


load d:\projets_python_ssd\perso\MH_Wilds_tools\.venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load d:\projets_python_ssd\perso\MH_Wilds_tools\.venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load d:\projets_python_ssd\perso\MH_Wilds_tools\.venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load d:\projets_python_ssd\perso\MH_Wilds_tools\.venv\Lib\site-packages\ortools\.libs\re2.dll...
load d:\projets_python_ssd\perso\MH_Wilds_tools\.venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load d:\projets_python_ssd\perso\MH_Wilds_tools\.venv\Lib\site-packages\ortools\.libs\highs.dll...
load d:\projets_python_ssd\perso\MH_Wilds_tools\.venv\Lib\site-packages\ortools\.libs\ortools.dll...


In [2]:
weapon_name = weapons.filter(pl.col("name") == "Lame d'espoir").to_dicts()[0]
talent_list = [
    {"name": "Tyrannie du Gore Magala", "target_level": 2, "weight": 1},
    {"name": "Volonté de l'Anjanath tonnerre", "target_level": 2, "weight": 1},
    {"name": "Fenêtre d'invulnérabilité", "target_level": -1, "weight": 1},
    {"name": "Poussée d'adrénaline", "target_level": -1, "weight": 1},
]
solution = solve(weapon_name, talent_list)


ic| weapon_name: "Lame d'espoir"
ic| talent_list: [{'name': 'Tyrannie du Gore Magala', 'target_level': 2, 'weight': 1},
                  {'name': "Volonté de l'Anjanath tonnerre", 'target_level': 2, 'weight': 1},
                  {'name': "Fenêtre d'invulnérabilité", 'target_level': -1, 'weight': 1},
                  {'name': "Poussée d'adrénaline", 'target_level': -1, 'weight': 1}]
ic| wanted_talents: {"Fenêtre d'invulnérabilité": 1,
                     "Poussée d'adrénaline": 1,
                     'Tyrannie du Gore Magala': 1,
                     "Volonté de l'Anjanath tonnerre": 1}
ic| f"Solver status: {solver_statuses[status]}": 'Solver status: OPTIMAL'
ic| _vars["talent_sums_final"]["Tyrannie du Gore Magala"]: talent_sums_final_Tyrannie du Gore Magala(0..30)
ic| _vars["talent_sums_final"]["Volonté de l'Anjanath tonnerre"]: talent_sums_final_Volonté de l'Anjanath tonnerre(0..30)
ic| model.model_stats(): ("optimization model '': (model_fingerprint: 0xd7845b68df5956f7)
       

In [47]:
solution

{'jewels': {'Joyau invulnérabilité [2]': 3, 'Joyau riposte [3]': 1},
 'weapon': "Lame d'espoir",
 'Bras': 'Avant-bras tonnerre G β',
 'Jambes': 'Grèves Gore Magala β',
 'Taille': 'Tassette tonnerre G β',
 'Torse': 'Cotte Gore Magala β',
 'Tête': 'Heaume Xu Wu β',
 'charm': 'Talisman de riposte II'}

In [34]:
solution_jewels = solution["jewels"]
pl.DataFrame(
    [{"name": key, "quantity": value} for key, value in solution_jewels.items()]
).join(jewels, on="name", how="inner").explode("jewel_talent_list").with_columns(
    pl.col("jewel_talent_list").struct.field("name").alias("talent_name"),
    pl.col("jewel_talent_list").struct.field("lvl").alias("talent_lvl"),
).with_columns(
    (pl.col("talent_lvl") * pl.col("quantity")).alias("Joyaux"),
).select("talent_name", "Joyaux")


talent_name,Joyaux
str,i64
"""Poussée d'adrénaline""",1
"""Fenêtre d'invulnérabilité""",3


variable,value
str,i64
"""jewel_1""",0
"""jewel_2""",0
"""jewel_3""",0


In [ ]:
jewel_types = (
    jewels
    #
    .explode("jewel_talent_list")
    .select(
        "name",
        pl.col("jewel_talent_list").struct.field("name").alias("jewel_talent_name"),
    )
    .join(
        talents.select(pl.col("name").alias("jewel_talent_name"), "group").unique(),
        on="jewel_talent_name",
        how="left",
    )
)


name,jewel_talent_name,group
str,str,str
"""Joyau attaque [1]""","""Machine de guerre""","""Weapon"""
"""Joyau attaque II [2]""","""Machine de guerre""","""Weapon"""
"""Joyau attaque III [3]""","""Machine de guerre""","""Weapon"""
"""Joyau vengeance [2]""","""Vengeance""","""Equip"""
"""Joyau riposte [3]""","""Poussée d'adrénaline""","""Equip"""
…,…,…
"""Joyau flash [1]""","""Aveuglement""","""Equip"""
"""Joyau botaniste [1]""","""Botaniste""","""Equip"""
"""Joyau géologie [1]""","""Géologiste""","""Equip"""


In [ ]:
from IPython.display import display, Markdown


solution_jewels = solution["jewels"]


jewel_types = (
    jewels
    #
    .explode("jewel_talent_list")
    .select(
        "name",
        pl.col("jewel_talent_list").struct.field("name").alias("jewel_talent_name"),
    )
    .join(
        talents.select(pl.col("name").alias("jewel_talent_name"), "group").unique(),
        on="jewel_talent_name",
        how="left",
    )
)


armor_slots = (
    pl.DataFrame(
        [
            {"name": solution[_type], "type": _type, "order": order}
            for order, _type in enumerate(["Tête", "Torse", "Bras", "Taille", "Jambes"])
        ],
    )
    #
    .join(
        armor,
        on="name",
        how="inner",
    )
    .unique("name")
    .select(pl.col(f"jewel_{i}").sum() for i in range(1, 4))
    .unpivot()
    .sort("variable")
    .to_dicts()
)

weapon_slots = (
    weapons
    #
    .filter(pl.col("name") == weapon_name)
    .select(
        pl.col("jewels").struct.field(str(i)).alias(f"jewel_{str(i)}")
        for i in range(1, 4)
    )
    .unpivot()
    .sort("variable")
    .to_dicts()
)


md = "### **Joyaux armure** \n\n"
for row in armor_slots:
    size = row["variable"].replace("jewel_", "")
    at_least_one_jewel = False
    free_slots = row["value"]
    for name, quantity in solution_jewels.items():
        if name in jewel_types.filter(pl.col("group") == "Weapon")["name"].to_list():
            continue
        if (
            not at_least_one_jewel
            and quantity > 0
            and name.split("[")[-1].replace("]", "") == size
        ):
            md += f"**Joyaux taille {size}**,  slots: {free_slots}\n\n"
            at_least_one_jewel = True
        if name.split("[")[-1].replace("]", "") == size:
            md += f"* {name} x{quantity}\n\n"
            free_slots -= quantity
    if free_slots > 0:
        if not at_least_one_jewel:
            md += f"**Joyaux taille {size}**,  slots: {free_slots}\n\n"
        md += f"* *Libre: {free_slots}*\n\n"

md += "### **Joyaux arme** \n\n"
for row in weapon_slots:
    size = row["variable"].replace("jewel_", "")
    at_least_one_jewel = False
    free_slots = row["value"]
    for name, quantity in solution_jewels.items():
        if (
            name
            not in jewel_types.filter(pl.col("group") == "Weapon")["name"].to_list()
        ):
            continue
        if (
            not at_least_one_jewel
            and quantity > 0
            and name.split("[")[-1].replace("]", "") == size
        ):
            md += f"**Joyaux taille {size}**,  slots: {free_slots}\n\n"
            at_least_one_jewel = True
        else:
            continue
        if name.split("[")[-1].replace("]", "") == size:
            md += f"* {name} x{quantity}\n\n"
            free_slots -= quantity
    if free_slots > 0:
        md += f"* *Libre: {free_slots}*\n\n"
display(Markdown(md))


### **Joyaux armure** 

**Joyaux taille 1**,  slots: 4

* *Libre: 4*

**Joyaux taille 2**,  slots: 5

* Joyau invulnérabilité [2] x3

* *Libre: 2*

**Joyaux taille 3**,  slots: 2

* Joyau riposte [3] x1

* *Libre: 1*

### **Joyaux arme** 



In [ ]:
for name, quantity in solution_jewels.items():
    if name.split("[")[-1].replace("]", "")== taille:
        md  + = f"* {name} x{quantity}\n\n"


{'Joyau invulnérabilité [2]': 3, 'Joyau riposte [3]': 1}

In [77]:
weapon_name = solution["weapon"]
weapon_name = "Singha Sharanga"
(
    weapons
    #
    .filter(pl.col("name") == weapon_name)
    .explode("talents")
    .select(
        pl.col("talents").struct.field("name").alias("talent_name"),
        pl.col("talents").struct.field("lvl").alias(weapon_name),
    )
)


talent_name,Singha Sharanga
str,i64
"""Quatrième tir""",3


In [19]:
charm = solution["charm"]
charms.filter(pl.col("name") == charm).select("talent_name", "talent_lvl").rename(
    {"talent_lvl": charm}
)


talent_name,Talisman de riposte II
str,i64
"""Poussée d'adrénaline""",2


In [33]:
# def get_markdown_from_solution(solution: dict) -> str:
solution_jewels = solution["jewels"]
weapon_name = solution["weapon"]

(
    (
        df := pl.DataFrame(
            [
                {"name": solution[_type], "type": _type, "order": order}
                for order, _type in enumerate(
                    ["Tête", "Torse", "Bras", "Taille", "Jambes"]
                )
            ],
        )
        #
        .join(
            armor,
            on="name",
            how="inner",
        )
        .sort("order")
        .pivot(
            index="talent_name",
            on="name",
            values="talent_level",
            aggregate_function="sum",
        )
        .join(
            pl.DataFrame(
                [
                    {"name": key, "quantity": value}
                    for key, value in solution_jewels.items()
                ]
            )
            .join(jewels, on="name", how="inner")
            .explode("jewel_talent_list")
            .with_columns(
                pl.col("jewel_talent_list").struct.field("name").alias("talent_name"),
                pl.col("jewel_talent_list").struct.field("lvl").alias("talent_lvl"),
            )
            .with_columns(
                (pl.col("talent_lvl") * pl.col("quantity")).alias("Joyaux"),
            )
            .select("talent_name", "Joyaux"),
            on="talent_name",
            how="left",
        )
        .join(
            weapons
            #
            .filter(pl.col("name") == weapon_name)
            .explode("talents")
            .select(
                pl.col("talents").struct.field("name").alias("talent_name"),
                pl.col("talents").struct.field("lvl").alias(weapon_name),
            ),
            on="talent_name",
            how="outer",
        )
        .with_columns(pl.coalesce("talent_name", "talent_name_right"))
        .drop("talent_name_right")
        .join(
            charms.filter(pl.col("name") == charm)
            .select("talent_name", "talent_lvl")
            .rename({"talent_lvl": charm}),
            on="talent_name",
            how="outer",
        )
        .with_columns(pl.coalesce("talent_name", "talent_name_right"))
        .drop("talent_name_right")
    )
    .with_columns(pl.lit(0).alias("Total"))
    .fill_null(0)
    .with_columns(
        pl.struct(df.columns[1:])
        .alias("Total")
        .map_elements(lambda s: sum(s.values()))
        .alias("Total")
    )
    .with_columns(pl.col(col).cast(pl.String).alias(col) for col in df.columns)
    .with_columns(pl.col(col).str.replace("0", "") for col in df.columns)
    .with_columns(pl.col("Joyaux").str.replace("0", ""))
    .sort("Total", "talent_name", descending=[True, False])
    .rename({"talent_name": "Talent"})
    .to_pandas()
    .to_markdown(index=False)
)


C:\Users\david\AppData\Local\Temp\ipykernel_50972\4157104420.py:48: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(
C:\Users\david\AppData\Local\Temp\ipykernel_50972\4157104420.py:62: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(
C:\Users\david\AppData\Local\Temp\ipykernel_50972\4157104420.py:74: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(


"| Talent                         | Heaume Xu Wu β   | Cotte Gore Magala β   | Avant-bras tonnerre G β   | Tassette tonnerre G β   | Grèves Gore Magala β   | Joyaux   | Lame d'espoir   | Talisman de riposte II   |   Total |\n|:-------------------------------|:-----------------|:----------------------|:--------------------------|:------------------------|:-----------------------|:---------|:----------------|:-------------------------|--------:|\n| Fenêtre d'invulnérabilité      |                  | 2                     |                           |                         |                        | 3        |                 |                          |       5 |\n| Poussée d'adrénaline           | 2                |                       |                           |                         |                        | 1        |                 | 2                        |       5 |\n| Protection du Gardien          |                  |                       | 1                        

| Talent                         | Heaume Arkvulcan β   | Cotte tonnerre G β   | Avant-bras tonnerre G β   | Braconnière Gore Magala β   | Grèves Gore Magala β   |   Total |
|:-------------------------------|:---------------------|:---------------------|:--------------------------|:----------------------------|:-----------------------|--------:|
| Athlète                        |                      |                      |                           | 2                           |                        |       2 |
| Corps et âme                   |                      | 1                    | 1                         |                             |                        |       2 |
| Protection du Gardien          |                      | 1                    | 1                         |                             |                        |       2 |
| Tyrannie du Gore Magala        |                      |                      |                           | 1                           | 1                      |       2 |
| Volonté de l'Anjanath tonnerre |                      | 1                    | 1                         |                             |                        |       2 |
| Écailles superposées           |                      |                      |                           | 1                           | 1                      |       2 |
| Antivirus                      |                      |                      |                           |                             | 1                      |       1 |
| Appétit de l'Arkveld           | 1                    |                      |                           |                             |                        |       1 |
| Conversion élémentaire         | 1                    |                      |                           |                             |                        |       1 |
| Nerfs d'acier                  |                      |                      |                           |                             | 1                      |       1 |
| Pelage attrayant               | 1                    |                      |                           |                             |                        |       1 |

In [5]:
talents.filter(pl.col("name") == "Volonté de l'Anjanath tonnerre")

group,name,description,href,levels
str,str,str,str,list[struct[2]]
"""Series""","""Volonté de l'Anjanath tonnerre""","""2 → Confère une jauge d'endura…","""https://mhwilds.kiranico.com/f…","[{2,""Confère une jauge d'endurance supplémentaire.""}, {4,""Confère une grande jauge d'endurance supplémentaire.""}]"


In [2]:
armor.filter(pl.col("talent_name") == "Volonté de l'Anjanath tonnerre")

piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Tête""","""Heaume tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,3,0,0,0,0
"""Torse""","""Cotte tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,2,0,1,0,0
"""Bras""","""Avant-bras tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,2,1,0,0,0
"""Taille""","""Tassette tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,1,2,0,0,0
"""Jambes""","""Grèves tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,2,1,0,0,0
"""Tête""","""Heaume tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,2,0,1,0,0
"""Torse""","""Cotte tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,0,2,1,0,0
"""Bras""","""Avant-bras tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,1,0,2,0,0
"""Taille""","""Tassette tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,1,0,2,0,0


In [ ]:
model = cp_model.CpModel()

use_equipement_vars = {}
talents_not_aggregated = {}
talents_aggregated = {}

use_equipement_vars["charm"] = {}

# Create non aggregated talent list
for charm in charms.iter_rows():
    charm_name, charm_href, talent_name, talent_level = charm

    use_charm = model.NewBoolVar(f"use_charm_{charm_name}")

    charm_talent_level = model.NewIntVar(
        lb=0,
        ub=4,
        name=f"charm_{charm_name}_talent_{talent_name}_level_{talent_level}",
    )

    model.Add(charm_talent_level == talent_level).only_enforce_if(use_charm)
    model.Add(charm_talent_level == 0).only_enforce_if(use_charm.Not())

    # Store variables
    use_equipement_vars["charm"][charm_name] = use_charm

    if talents_not_aggregated.get(talent_name) is None:
        talents_not_aggregated[talent_name] = []
    talents_not_aggregated[talent_name].append(charm_talent_level)

# Compute talent sums into proxy variables
for talent_name, talent_lvl_list in talents_not_aggregated.items():
    talent_sum = model.NewIntVar(0, 4, f"talent_sum_{talent_name}")

    # Map the sum
    model.Add(talent_sum == sum(talent_lvl_list))
    talents_aggregated[talent_name] = talent_sum

# Say that there is at most one charm
model.Add(sum(list(use_equipement_vars["charm"].values())) <= 1)

In [ ]:
single_objective = talents_aggregated["Vengeance"]
# single_objective = talents_aggregated["Auto-amélioration"]

model.maximize(single_objective)
solver = cp_model.CpSolver()
status = solver.solve(model)

solver_statuses = {
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"Solver status: {solver_statuses[status]}")
# if status in (cp_model.OPTIMAL, status == cp_model.FEASIBLE):
#     print(solver.value(single_objective))
for charm_name, use_charm_variable in use_equipement_vars["charm"].items():
    if solver.value(use_charm_variable) == 1:
        print(charm_name)

Solver status: OPTIMAL
Talisman de fureur II


In [18]:
armor

piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Tête""","""Masque d'espoir""","""Crâne d'acier""",1,3,0,0,0,0
"""Torse""","""Cotte d'espoir""","""Bénédiction""",1,3,0,0,0,0
"""Bras""","""Avant-bras d'espoir""","""Bénédiction""",1,3,0,0,0,0
"""Taille""","""Tassette d'espoir""","""Crâne d'acier""",1,3,0,0,0,0
"""Jambes""","""Grèves d'espoir""","""Bénédiction""",1,3,0,0,0,0
…,…,…,…,…,…,…,…,…
"""Tête""","""Mimiflore α""","""Pelage de renforcement""",1,2,1,0,0,0
"""Tête""","""Mimiflore α""","""Entomologiste""",1,2,1,0,0,0
"""Tête""","""Mimiflore α""","""Embuscade""",1,2,1,0,0,0


In [ ]:
talents_lvl_max = (
    talents
    #
    .explode("levels")
    .with_columns(
        pl.col("levels").struct.field("lvl").alias("talent_lvl"),
        pl.col("levels").struct.field("description").alias("talent_description"),
    )
    .filter(pl.col("talent_lvl") == pl.col("talent_lvl").max().over("name"))
    .select("group", "name", "talent_lvl")
)
talents_lvl_max

group,name,talent_lvl
str,str,i64
"""Weapon""","""Machine de guerre""",5
"""Weapon""","""Garde offensive""",3
"""Weapon""","""Maître d'armes""",5
"""Weapon""","""Berserker""",5
"""Weapon""","""Dégainage éclair""",3
…,…,…
"""Series""","""Mutinerie du Nu Udra""",4
"""Series""","""Révolte du Jin Dahaad""",4
"""Series""","""Tyrannie du Gore Magala""",4


In [ ]:
weapon_name = "Brisefoi Leibolkule"

model = cp_model.CpModel()
_vars = {
    "use_armor_piece_booleans": {},
    "use_charm_booleans": {},
    "talent_lists": {},
    "talent_sums": {},
    "talent_sums_capped": {},
    "jewel_emplacement_lists": {},
    "jewel_emplacement_sums": {},
    "jewel_emplacement_sums_total_armor": {},
    "jewel_emplacement_sums_total_weapon": {},
    "jewel_uses_integers": {},
}

unique_pieces = armor["piece"].unique().sort().to_list()

for armor_piece in unique_pieces:
    if _vars.get("use_armor_piece_booleans").get(armor_piece) is None:
        _vars["use_armor_piece_booleans"][armor_piece] = {}

    if _vars.get("jewel_emplacement_lists").get(armor_piece) is None:
        _vars["jewel_emplacement_lists"][armor_piece] = {}

    if _vars.get("jewel_emplacement_sums").get(armor_piece) is None:
        _vars["jewel_emplacement_sums"][armor_piece] = {}

    armor_piece_filtered = armor.filter(pl.col("piece") == armor_piece)
    unique_armor_pieces_names = armor_piece_filtered["name"].unique().sort().to_list()
    for unique_armor_piece_name in unique_armor_pieces_names:
        unique_armor_piece = armor_piece_filtered.filter(
            pl.col("name") == unique_armor_piece_name
        )

        # Define a boolean that tells if the armor piece is equipped
        names = unique_armor_piece["name"].to_list()
        name = names[0]
        armor_piece_equipped = model.NewBoolVar(f"use_piece_{armor_piece}_{name}")

        for row in unique_armor_piece.iter_rows():
            (
                _,
                name,
                talent_name,
                talent_level,
                jewel_0,
                jewel_1,
                jewel_2,
                jewel_3,
                jewel_4,
            ) = row
            # Create a variable that tells that the talent is active due to the fact that the armor piece is equipped
            var_talent_lvl = model.NewIntVar(
                lb=0,
                ub=30,
                name=f"talent_{talent_name}_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_talent_lvl == talent_level).only_enforce_if(
                armor_piece_equipped
            )
            model.Add(var_talent_lvl == 0).only_enforce_if(armor_piece_equipped.Not())

            # Create the key if it doesn't exist
            if _vars["talent_lists"].get(talent_name) is None:
                _vars["talent_lists"][talent_name] = []

            # Jewel emplacement part
            # LVL 1
            var_nb_jewel_1 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_1_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_1 == jewel_1).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_1 == 0).only_enforce_if(armor_piece_equipped.Not())

            # LVL 2
            var_nb_jewel_2 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_2_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_2 == jewel_2).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_2 == 0).only_enforce_if(armor_piece_equipped.Not())

            # LVL 3
            var_nb_jewel_3 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_3_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_3 == jewel_3).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_3 == 0).only_enforce_if(armor_piece_equipped.Not())

            # LVL 4
            var_nb_jewel_4 = model.NewIntVar(
                lb=0,
                ub=4,
                name=f"jewel_lvl_4_from_type_{armor_piece}_with_{name}",
            )
            model.Add(var_nb_jewel_4 == jewel_4).only_enforce_if(armor_piece_equipped)
            model.Add(var_nb_jewel_4 == 0).only_enforce_if(armor_piece_equipped.Not())

            # Register variables
            _vars["talent_lists"][talent_name].append(var_talent_lvl)

        if _vars.get("jewel_emplacement_lists").get(armor_piece) is None:
            _vars["jewel_emplacement_lists"][armor_piece] = {}

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl1") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl1"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl1"].append(var_nb_jewel_1)

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl2") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl2"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl2"].append(var_nb_jewel_2)

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl3") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl3"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl3"].append(var_nb_jewel_3)

        if _vars.get("jewel_emplacement_lists").get(armor_piece).get("lvl4") is None:
            _vars["jewel_emplacement_lists"][armor_piece]["lvl4"] = []
        _vars["jewel_emplacement_lists"][armor_piece]["lvl4"].append(var_nb_jewel_4)

        # Store the boolean that tells if the armor piece is equipped
        _vars["use_armor_piece_booleans"][armor_piece][name] = armor_piece_equipped

    # Create variables that are the sum of the jewel emplacements
    for i in range(1, 5):
        lvl = f"lvl{i}"
        var_jewel_emplacement_sums = model.NewIntVar(
            lb=0,
            ub=30,
            name=f"jewel_emplacement_sums_{lvl}_from_type_{armor_piece}",
        )
        model.Add(
            var_jewel_emplacement_sums
            == sum(_vars["jewel_emplacement_lists"][armor_piece][lvl])
        )
        _vars["jewel_emplacement_sums"][armor_piece][lvl] = var_jewel_emplacement_sums

    # Add the constraint of only one type of armor piece equipped at a time
    model.Add(sum(_vars["use_armor_piece_booleans"][armor_piece].values()) <= 1)

# Create a variable that symbolizes the total number of armor jewels
for i in range(1, 5):
    lvl = f"lvl{i}"
    var_jewel_emplacement_sums = model.NewIntVar(
        lb=0,
        ub=30,
        name=f"jewel_{lvl}_emplacement_sums_for_all_armor_pieces",
    )
    model.Add(
        var_jewel_emplacement_sums
        == sum(
            _vars["jewel_emplacement_sums"][armor_piece][lvl]
            for armor_piece in unique_pieces
        )
    )
    _vars["jewel_emplacement_sums_total_armor"][lvl] = var_jewel_emplacement_sums

# Charm talents part
unique_charm_name = charms["name"].unique().sort().to_list()
for charm_name in unique_charm_name:
    charm_data = charms.filter(pl.col("name") == charm_name).to_dicts()

    use_charm_var = model.NewBoolVar(f"use_charm_{charm_name}")
    for row in charm_data:
        charm_name = row["name"]
        charm_talent = row["talent_name"]
        charm_lvl = row["talent_lvl"]

        charm_talent_lvl = model.NewIntVar(
            lb=0,
            ub=30,
            name=f"charm_talent_{talent_name}_lvl_{charm_lvl}_from_{charm_name}",
        )
        model.Add(charm_talent_lvl == charm_lvl).only_enforce_if(use_charm_var)
        model.Add(charm_talent_lvl == 0).only_enforce_if(use_charm_var.Not())

        # Register the variable
        if _vars["talent_lists"].get(charm_talent) is None:
            _vars["talent_lists"][charm_talent] = []

        _vars["talent_lists"][charm_talent].append(charm_talent_lvl)
    _vars["use_charm_booleans"][charm_name] = use_charm_var

# Add the constraint of only one charm equipped at a time
model.Add(sum(_vars["use_charm_booleans"].values()) <= 1)

# Weapon talent part
weapon = (
    weapons.filter(pl.col("name") == weapon_name)
    .explode("talents")
    .select(
        "name",
        *[
            pl.col("jewels").struct.field(str(i)).alias(f"jewel_lvl{i}")
            for i in range(4)
        ],
        pl.col("talents").struct.field("name").alias("talent_name"),
        pl.col("talents").struct.field("lvl").alias("talent_lvl"),
    )
)
for row in weapon.to_dicts():
    talent_name = row["talent_name"]
    talent_lvl = row["talent_lvl"]
    var_talent_lvl = model.NewIntVar(
        lb=1,
        ub=30,
        name=f"weapon_talent_{talent_name}_lvl_{talent_lvl}",
    )
    model.Add(var_talent_lvl == talent_lvl)
    # Register the variable
    if _vars["talent_lists"].get(talent_name) is None:
        _vars["talent_lists"][talent_name] = []
    _vars["talent_lists"][talent_name].append(var_talent_lvl)

# Create a variable that symbolizes the total number of weapon jewels
for i in range(4):
    lvl = f"lvl{i}"
    var_jewel_emplacement_sums = model.NewIntVar(
        lb=0,
        ub=30,
        name=f"jewel_emplacement_sums_{lvl}_from_weapon",
    )
    model.Add(var_jewel_emplacement_sums == row[f"jewel_{lvl}"])
    # Register the variable
    _vars["jewel_emplacement_sums_total_weapon"][lvl] = var_jewel_emplacement_sums


# Jewels
## Register the number of jewels used
all_jewels = jewels.explode("jewel_talent_list").select(
    pl.col("name").alias("jewel_name"),
    "jewel_lvl",
    pl.col("jewel_talent_list").struct.field("name").alias("talent_name"),
    pl.col("jewel_talent_list").struct.field("lvl").alias("talent_lvl"),
)
unique_jewels_names = all_jewels["jewel_name"].unique().sort().to_list()
for jewel_name in unique_jewels_names:
    jewel_data = all_jewels.filter(pl.col("jewel_name") == jewel_name)
    nb_of_jewel_use = model.NewIntVar(lb=0, ub=100, name=f"nb_of_use_of_{jewel_name}")
    for row in jewel_data.to_dicts():
        talent_name = row["talent_name"]
        talent_lvl = row["talent_lvl"]
        total_talent = model.NewIntVar(
            lb=0,
            ub=100,
            name=f"total_talent_{talent_name}_lvl_of_for_jewel_{jewel_name}",
        )
        model.Add(total_talent == nb_of_jewel_use * talent_lvl)

        if talent_name not in _vars["talent_lists"]:
            _vars["talent_lists"][talent_name] = []
        _vars["talent_lists"][talent_name].append(total_talent)
    jewel_lvl = row["jewel_lvl"]
    if jewel_lvl not in _vars["jewel_uses_integers"]:
        _vars["jewel_uses_integers"][jewel_lvl] = {}
    _vars["jewel_uses_integers"][jewel_lvl][jewel_name] = nb_of_jewel_use

# Add contraints for maximum number of jewels
## Get jewel types
jewel_types = all_jewels.join(
    talents.select("group", pl.col("name").alias("talent_name")).unique(),
    on="talent_name",
)

## Get armor jewel types
armor_jewel_types = jewel_types.filter(pl.col("group") == "Equip")
# Jewels 1
nb_of_uses_of_jewel1 = sum(
    var
    for jewel_name, var in _vars["jewel_uses_integers"][1].items()
    if jewel_name in armor_jewel_types["jewel_name"].unique().to_list()
)
# Jewels 1 can fit in lvl 1, 2, 3
model.Add(
    nb_of_uses_of_jewel1
    <= sum(_vars["jewel_emplacement_sums_total_armor"][f"lvl{i}"] for i in range(1, 4))
)

# Jewels 2
nb_of_uses_of_jewel2 = sum(
    var
    for jewel_name, var in _vars["jewel_uses_integers"][2].items()
    if jewel_name in armor_jewel_types["jewel_name"].unique().to_list()
)
# Jewels 2 can fit in lvl 2, 3
model.Add(
    nb_of_uses_of_jewel2
    <= sum(_vars["jewel_emplacement_sums_total_armor"][f"lvl{i}"] for i in range(2, 4))
)

# Jewels 3
nb_of_uses_of_jewel3 = sum(
    var
    for jewel_name, var in _vars["jewel_uses_integers"][3].items()
    if jewel_name in armor_jewel_types["jewel_name"].unique().to_list()
)
# Jewels 3 can only fit in lvl 3
model.Add(nb_of_uses_of_jewel3 <= _vars["jewel_emplacement_sums_total_armor"]["lvl3"])

## Get weapon jewel types
weapon_jewel_types = jewel_types.filter(pl.col("group") == "Weapon")
# Jewels 1
nb_of_uses_of_jewel1 = sum(
    var
    for jewel_name, var in _vars["jewel_uses_integers"][1].items()
    if jewel_name in weapon_jewel_types["jewel_name"].unique().to_list()
)
# Jewels 1 can fit in lvl 1, 2, 3
model.Add(
    nb_of_uses_of_jewel1
    <= sum(_vars["jewel_emplacement_sums_total_weapon"][f"lvl{i}"] for i in range(1, 4))
)

# Jewels 2
nb_of_uses_of_jewel2 = sum(
    var
    for jewel_name, var in _vars["jewel_uses_integers"][2].items()
    if jewel_name in weapon_jewel_types["jewel_name"].unique().to_list()
)
# Jewels 2 can fit in lvl 2, 3
model.Add(
    nb_of_uses_of_jewel2
    <= sum(_vars["jewel_emplacement_sums_total_weapon"][f"lvl{i}"] for i in range(2, 4))
)

# Jewels 3
nb_of_uses_of_jewel3 = sum(
    var
    for jewel_name, var in _vars["jewel_uses_integers"][3].items()
    if jewel_name in weapon_jewel_types["jewel_name"].unique().to_list()
)
# Jewels 3 can only fit in lvl 3
model.Add(nb_of_uses_of_jewel3 <= _vars["jewel_emplacement_sums_total_weapon"]["lvl3"])


# Compute the talent sums
for talent_name, talent_vars in _vars["talent_lists"].items():
    var_talent_sum = model.NewIntVar(
        lb=0,
        ub=30,
        name=f"talent_sum_for_{talent_name}",
    )
    model.Add(var_talent_sum == sum(talent_vars))

    _vars["talent_sums"][talent_name] = var_talent_sum

# Add talent sum cap
unique_talent_names = talents["name"].unique().to_list()
talents_lvl_max = (
    talents
    #
    .explode("levels")
    .with_columns(
        pl.col("levels").struct.field("lvl").alias("talent_lvl"),
        pl.col("levels").struct.field("description").alias("talent_description"),
    )
    .filter(pl.col("talent_lvl") == pl.col("talent_lvl").max().over("name"))
    .select("group", "name", "talent_lvl")
)
for talent_name in unique_talent_names:
    # Get the talent max lvl
    individual_max_level = talents_lvl_max.filter(
        pl.col("name") == talent_name
    ).to_dicts()[0]

    var_talent_sum_capped = model.NewIntVar(
        lb=0,
        ub=30,
        name=f"talent_sum_capped_for_{talent_name}",
    )
    model.AddMinEquality(
        target=var_talent_sum_capped,
        exprs=[_vars["talent_sums"][talent_name], individual_max_level["talent_lvl"]],
    )
    # Store the capped talent sum
    _vars["talent_sums_capped"][talent_name] = var_talent_sum_capped


# Set bonus talents
group_talent_names = (
    talents.filter(pl.col("group") == "Group")["name"].unique().to_list()
)
for group_talent_name in group_talent_names:
    data = (
        talents
        #
        .filter(pl.col("name") == group_talent_name)
        .explode("levels")
        .with_columns(pl.col("levels").struct.field("lvl").alias("talent_lvl"))
    ).to_dicts()[0]
    # If the number of group talent is below the lvl, set it to 0
    group_has_enough_levels = model.NewBoolVar(
        name=f"group_talent_has_enough_levels_{group_talent_name}"
    )
    model.Add(
        _vars["talent_sums_capped"][group_talent_name] >= data["talent_lvl"]
    ).OnlyEnforceIf(group_has_enough_levels)
    model.Add(_vars["talent_sums_capped"][group_talent_name] == 0).OnlyEnforceIf(
        group_has_enough_levels.Not()
    )

# Add set series bonus talents
talent_series = (
    talents.filter(pl.col("group") == "Series")
    .explode("levels")
    .select(
        "name",
        pl.col("levels").struct.field("lvl").alias("talent_lvl"),
    )
)
talent_series_names = talent_series["name"].unique().to_list()
for talent_series_name in talent_series_names:
    # Filter the talent series for the current series name
    unique_serie = talent_series.filter(pl.col("name") == talent_series_name)
    # Create a list of levels, starting with 0
    levels = [0] + unique_serie.sort("talent_lvl")["talent_lvl"].to_list()

    # Iterate over pairs of consecutive levels
    for talent_inferior, talent_superior in pairwise(levels):
        # Create a boolean variable to check if the talent level is more than the inferior level
        var_talent_lvl_is_more_than_inferior = model.NewBoolVar(
            name=f"talent_series_{talent_series_name}_greater_than_{talent_inferior}"
        )
        # Add constraints based on the boolean variable
        model.Add(
            talent_inferior <= _vars["talent_sums_capped"][talent_series_name]
        ).only_enforce_if(var_talent_lvl_is_more_than_inferior)
        model.Add(
            talent_inferior > _vars["talent_sums_capped"][talent_series_name]
        ).only_enforce_if(var_talent_lvl_is_more_than_inferior.Not())

        # Create a boolean variable to check if the talent level is less than the superior level
        var_talent_lvl_is_less_than_superior = model.NewBoolVar(
            name=f"talent_series_{talent_series_name}_less_than_{talent_superior}"
        )
        # Add constraints based on the boolean variable
        model.Add(
            talent_superior > _vars["talent_sums_capped"][talent_series_name]
        ).only_enforce_if(var_talent_lvl_is_less_than_superior)
        model.Add(
            talent_superior <= _vars["talent_sums_capped"][talent_series_name]
        ).only_enforce_if(var_talent_lvl_is_less_than_superior.Not())

        # Create a boolean variable to check if the talent level is between the inferior and superior levels
        var_talent_lvl_is_between = model.NewBoolVar(
            name=f"talent_series_{talent_series_name}_between_{talent_inferior}_and_{talent_superior}"
        )
        # Add constraints to enforce the 'between' condition
        model.Add(
            sum(
                [
                    var_talent_lvl_is_more_than_inferior,
                    var_talent_lvl_is_less_than_superior,
                ]
            )
            == 2
        ).only_enforce_if(var_talent_lvl_is_between)
        model.Add(
            sum(
                [
                    var_talent_lvl_is_more_than_inferior,
                    var_talent_lvl_is_less_than_superior,
                ]
            )
            < 2
        ).only_enforce_if(var_talent_lvl_is_between.Not())

        # Add a constraint to set the talent level to the inferior level if the 'between' condition is met
        model.Add(
            _vars["talent_sums_capped"][talent_series_name] == talent_inferior
        ).only_enforce_if(var_talent_lvl_is_between)


## Objective
wanted_talents = {
    "Cercle de vie": 1,
    "Halo de guérison": 1,
}

# Add additionnal value for free jewels emplacements
lvl_emplacements = {}
for lvl in range(1, 4):
    lvl_emplacements[lvl] = []
    for armor_piece, _dict in _vars["jewel_emplacement_sums"].items():
        lvl_emplacements[lvl].append(_dict[f"lvl{lvl}"])

maximize_nb_of_free_jewels = []
maximize_nb_of_free_jewels.extend(
    sum(lvl_emplacements[lvl])
    - sum(_vars["jewel_uses_integers"][lvl].values()) * 10**3 * 10**lvl
    for lvl in range(1, 4)
)
# Add additional objective value for eventual addional talents (still minimize nb of jewels)
nb_of_talents = sum(_vars["talent_sums_capped"].values())


# Add optional strict talent optimization
talent_objective = (
    sum(
        _vars["talent_sums_capped"][talent] * 10**talent_weight
        for talent, talent_weight in wanted_talents.items()
    )
    * 10**9
)


model.maximize(
    sum(
        [
            talent_objective,
            sum(maximize_nb_of_free_jewels),
        ]
    )
)
solver = cp_model.CpSolver()
status = solver.solve(model)

solver_statuses = {
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.UNKNOWN: "UNKNOWN",
}

print(f"Solver status: {solver_statuses[status]}")
solution = {"jewels": {}, "weapon": weapon.to_dicts()[0]["name"]}
for armor_piece, var_dict in _vars["use_armor_piece_booleans"].items():
    for name, var in var_dict.items():
        if solver.value(var) == 1:
            solution[armor_piece] = name
            display(armor.filter(pl.col("name") == name))
for charm, var in _vars["use_charm_booleans"].items():
    if solver.value(expression=var) == 1:
        solution["charm"] = charm
        display(charms.filter(pl.col("name") == charm))
for jewel_lvl, var_dict in _vars["jewel_uses_integers"].items():
    for jewel_name, var in var_dict.items():
        if solver.value(var) > 0:
            solution["jewels"][jewel_name] = solver.value(var)

solution

Solver status: OPTIMAL


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Bras""","""Avant-bras Arkvulcan β""","""Appétit de l'Arkveld""",1,0,1,2,0,0
"""Bras""","""Avant-bras Arkvulcan β""","""Pelage attrayant""",1,0,1,2,0,0
"""Bras""","""Avant-bras Arkvulcan β""","""Conversion élémentaire""",1,0,1,2,0,0


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Jambes""","""Grèves Doshaguma β""","""Puissance du Doshaguma""",1,0,2,1,0,0
"""Jambes""","""Grèves Doshaguma β""","""Pelage attrayant""",1,0,2,1,0,0
"""Jambes""","""Grèves Doshaguma β""","""Pique-assiette""",1,0,2,1,0,0
"""Jambes""","""Grèves Doshaguma β""","""Cercle de vie""",1,0,2,1,0,0


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Taille""","""Tassette Arkvulcan β""","""Appétit de l'Arkveld""",1,1,2,0,0,0
"""Taille""","""Tassette Arkvulcan β""","""Pelage attrayant""",1,1,2,0,0,0
"""Taille""","""Tassette Arkvulcan β""","""Mise à mort""",2,1,2,0,0,0
"""Taille""","""Tassette Arkvulcan β""","""Halo de guérison""",1,1,2,0,0,0


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Torse""","""Cotte Damascus α""","""Un pour tous""",1,1,0,2,0,0
"""Torse""","""Cotte Damascus α""","""Cercle de vie""",2,1,0,2,0,0


piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Tête""","""Heaume Doshaguma β""","""Puissance du Doshaguma""",1,0,2,1,0,0
"""Tête""","""Heaume Doshaguma β""","""Pelage attrayant""",1,0,2,1,0,0
"""Tête""","""Heaume Doshaguma β""","""Pique-assiette""",1,0,2,1,0,0
"""Tête""","""Heaume Doshaguma β""","""Crâne d'acier""",1,0,2,1,0,0


name,href,talent_name,talent_lvl
str,str,str,i64
"""Talisman de guérison III""","""https://mhwilds.kiranico.com/f…","""Halo de guérison""",3


{'jewels': {},
 'weapon': 'Brisefoi Leibolkule',
 'Bras': 'Avant-bras Arkvulcan β',
 'Jambes': 'Grèves Doshaguma β',
 'Taille': 'Tassette Arkvulcan β',
 'Torse': 'Cotte Damascus α',
 'Tête': 'Heaume Doshaguma β',
 'charm': 'Talisman de guérison III'}

In [ ]:
talents.filter(pl.col("group") == "Series").explode("levels").select(
    pl.col("name").alias("talent_name"),
    pl.col("levels").struct.field("lvl").alias("talent_level"),
)

talent_name,talent_level
str,i64
"""Puissance du Doshaguma""",2
"""Puissance du Doshaguma""",4
"""Éclat du Rathalos""",2
"""Éclat du Rathalos""",4
"""Vigueur du Xu Wu""",2
…,…
"""Tyrannie du Gore Magala""",4
"""Appétit de l'Arkveld""",2
"""Appétit de l'Arkveld""",4


In [ ]:
solution_talents = defaultdict(int)
for jewel_name, quantity in solution["jewels"].items():
    for row in (
        jewels.filter(pl.col("name") == jewel_name)
        .explode("jewel_talent_list")
        .select(
            pl.col("jewel_talent_list").struct.field("name").alias("talent_name"),
            pl.col("jewel_talent_list").struct.field("lvl").alias("talent_lvl"),
        )
        .to_dicts()
    ):
        solution_talents[row["talent_name"]] += row["talent_lvl"]
for row in (
    weapons.filter(pl.col("name") == solution["weapon"])
    .explode("talents")
    .select(
        pl.col("talents").struct.field("lvl").alias("talent_lvl"),
        pl.col("talents").struct.field("name").alias("talent_name"),
    )
    .to_dicts()
):
    solution_talents[row["talent_name"]] += row["talent_lvl"]

for armor_piece in armor["piece"].unique().sort().to_list():
    if armor_piece not in solution.keys():
        continue
    for row in armor.filter(pl.col("name") == solution[armor_piece]).to_dicts():
        solution_talents[row["talent_name"]] += row["talent_level"]

for row in charms.filter(pl.col("name") == solution["charm"]).to_dicts():
    solution_talents[row["talent_name"]] += row["talent_lvl"]

_temp = []
_temp.extend(
    {
        "talent_name": key,
        "talent_lvl": value,
    }
    for key, value in solution_talents.items()
)
df_solution_talents = pl.DataFrame(_temp).sort("talent_lvl", descending=True)
df_solution_talents.join(
    talents.explode("levels").select(
        pl.col("name").alias("talent_name"),
        pl.col("levels").struct.field("lvl").alias("talent_lvl"),
        pl.col("description").alias("general_description"),
        pl.col("levels").struct.field("description").alias("talent_description"),
    ),
    on=["talent_name", "talent_lvl"],
    how="inner",
)

talent_name,talent_lvl,general_description,talent_description
str,i64,str,str
"""Premier tir""",3,"""Augmente la vitesse de recharg…","""Augmente grandement la vitesse…"
"""Mise à mort""",2,"""Augmente l'affinité lorsque vo…","""Affinité +10 % pour les attaqu…"
"""Conversion élémentaire""",1,"""En subissant des dégâts élémen…","""Inflige des dégâts dragon addi…"
"""Crâne d'acier""",1,"""Réduit la durée d'étourdisseme…","""Durée étourdissement -30 %"""
"""Cercle de vie""",3,"""Augmente la quantité de vie re…","""Augmente grandement la récupér…"
"""Un pour tous""",1,"""Partage les effets de certains…","""Permet de partager 33 % des ef…"
"""Pique-assiette""",2,"""Octroie une chance prédéfinie …","""Chances d'activation +25 %"""
"""Puissance du Doshaguma""",2,"""2 → Octroie temporairement Att…","""Octroie temporairement Attaque…"
"""Appétit de l'Arkveld""",2,"""2 → Restaure une quantité modé…","""Restaure une quantité modérée …"


In [ ]:
talents.explode("levels").select(
    pl.col("name").alias("talent_name"),
    pl.col("levels").struct.field("lvl").alias("talent_lvl"),
    pl.col("description").alias("general_description"),
    pl.col("levels").struct.field("description").alias("talent_description"),
)

talent_name,talent_lvl,general_description,talent_description
str,i64,str,str
"""Machine de guerre""",1,"""Augmente l'attaque.""","""Attaque +3"""
"""Machine de guerre""",2,"""Augmente l'attaque.""","""Attaque +5"""
"""Machine de guerre""",3,"""Augmente l'attaque.""","""Attaque +7"""
"""Machine de guerre""",4,"""Augmente l'attaque.""","""Attaque +2 % Attaque +8"""
"""Machine de guerre""",5,"""Augmente l'attaque.""","""Attaque +4 % Attaque +9"""
…,…,…,…
"""Tyrannie du Gore Magala""",4,"""2 → Vous inflige la Furie face…","""Vous inflige la Furie et augme…"
"""Appétit de l'Arkveld""",2,"""2 → Restaure une quantité modé…","""Restaure une quantité modérée …"
"""Appétit de l'Arkveld""",4,"""2 → Restaure une quantité modé…","""Restaure de la vie tant que vo…"


In [ ]:
(
    armor
    #
    .filter(pl.col("talent_name").str.contains("Volonté de l'Anjanath tonnerre"))
)

piece,name,talent_name,talent_level,jewel_0,jewel_1,jewel_2,jewel_3,jewel_4
str,str,str,i64,i64,i64,i64,i64,i64
"""Tête""","""Heaume tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,3,0,0,0,0
"""Torse""","""Cotte tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,2,0,1,0,0
"""Bras""","""Avant-bras tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,2,1,0,0,0
"""Taille""","""Tassette tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,1,2,0,0,0
"""Jambes""","""Grèves tonnerre G α""","""Volonté de l'Anjanath tonnerre""",1,2,1,0,0,0
"""Tête""","""Heaume tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,2,0,1,0,0
"""Torse""","""Cotte tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,0,2,1,0,0
"""Bras""","""Avant-bras tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,1,0,2,0,0
"""Taille""","""Tassette tonnerre G β""","""Volonté de l'Anjanath tonnerre""",1,1,0,2,0,0


In [ ]:
model.Add()

In [ ]:
group_talent_names
group_talent_name = group_talent_names[0]
(
    talents
    #
    .filter(pl.col("name") == group_talent_name)
    .explode("levels")
    .with_columns(pl.col("levels").struct.field("lvl").alias("talent_lvl"))
)

group,name,description,href,levels,talent_lvl
str,str,str,str,struct[2],i64
"""Group""","""Écailles superposées""","""3 → Réduit temporairement la p…","""https://mhwilds.kiranico.com/f…","{3,""Réduit temporairement la perte d'endurance lorsque la vie est inférieure ou égale à 40 %.""}",3


In [ ]:
talents["name"].unique().to_list()

['Tension du Rey Dau',
 'Destructeur',
 'Recharge chargée',
 'Neutralisation',
 "Couverture de l'Uth Duna",
 'Performance optimale',
 'Témérité',
 'Camouflage Neopteron',
 'Pourfendeur de dragon',
 'Corps et âme',
 'Surcharge',
 'Main de maître',
 'Tir maximum',
 'Premier tir',
 "Nerfs d'acier",
 'Aura draconique',
 'Alerte Neopteron',
 'Vendetta',
 'Matraquage',
 'Force latente',
 'Pelage de renforcement',
 'Rage supérieure',
 'Puissance du Doshaguma',
 'Salve mortelle',
 'Totem élémentaire',
 'Expert en survie',
 'Désigné pour mourir',
 'Vigueur du Xu Wu',
 'Acier trempé',
 'Maelstrom',
 'Rage élémentaire',
 'Révolte du Jin Dahaad',
 'Métabolisme',
 'Protection du Gravios',
 'Apothicaire',
 'Union',
 'Pare-vent',
 'Sismologie',
 "Bouchon d'oreilles",
 "Volonté de l'Anjanath tonnerre",
 'Anti-immobilisation',
 'Berserker',
 'Blizzard',
 'Mise à mort',
 'Affûtage rapide',
 'Maître bombardier',
 "Poussée d'adrénaline",
 'Contre-attaque',
 'Insomnie',
 "Vol d'endurance",
 "Crâne d'acier"

In [ ]:
jewel_name = "Joyau chamanisme/glace [3]"
jewel_data = all_jewels.filter(pl.col("jewel_name") == jewel_name)
nb_of_jewel_use = model.NewIntVar(lb=0, ub=100, name=f"nb_of_use_of_{jewel_name}")
for row in jewel_data.to_dicts():
    talent_name = row["talent_name"]
    talent_lvl = row["talent_lvl"]
    total_talent = model.NewIntVar(
        lb=0,
        ub=100,
        name=f"total_talent_{talent_name}_lvl_of_for_jewel_{jewel_name}",
    )
    model.Add(total_talent == nb_of_jewel_use * talent_lvl)

    if talent_name not in _vars["talent_lists"]:
        _vars["talent_lists"][talent_name] = []
    _vars["talent_lists"][talent_name].append(total_talent)
jewel_lvl = row["jewel_lvl"]
if jewel_lvl not in _vars["jewel_uses_integers"]:
    _vars["jewel_uses_integers"][jewel_lvl] = {}
_vars["jewel_uses_integers"][jewel_lvl][jewel_name] = nb_of_jewel_use
_vars["jewel_uses_integers"]

{2: {'Joyau acrobatie [2]': nb_of_use_of_Joyau acrobatie [2](0..100),
  'Joyau aiguiseur II [2]': nb_of_use_of_Joyau aiguiseur II [2](0..100),
  'Joyau armurier II [2]': nb_of_use_of_Joyau armurier II [2](0..100),
  'Joyau artisanat II [2]': nb_of_use_of_Joyau artisanat II [2](0..100),
  'Joyau attaque II [2]': nb_of_use_of_Joyau attaque II [2](0..100),
  'Joyau berserk II [2]': nb_of_use_of_Joyau berserk II [2](0..100),
  'Joyau bouclier II [2]': nb_of_use_of_Joyau bouclier II [2](0..100),
  'Joyau chamanisme II [2]': nb_of_use_of_Joyau chamanisme II [2](0..100),
  'Joyau charge II [2]': nb_of_use_of_Joyau charge II [2](0..100),
  'Joyau châtiment II [2]': nb_of_use_of_Joyau châtiment II [2](0..100),
  'Joyau cogneur II [2]': nb_of_use_of_Joyau cogneur II [2](0..100),
  'Joyau concentration II [2]': nb_of_use_of_Joyau concentration II [2](0..100),
  'Joyau contre-attaque [2]': nb_of_use_of_Joyau contre-attaque [2](0..100),
  'Joyau destruction [2]': nb_of_use_of_Joyau destruction [2](

In [ ]:
_vars["jewel_uses_integers"][3]

{'Joyau acrobatie/artisanat [3]': nb_of_use_of_Joyau acrobatie/artisanat [3](0..100),
 'Joyau acrobatie/attaque [3]': nb_of_use_of_Joyau acrobatie/attaque [3](0..100),
 'Joyau acrobatie/dégainage [3]': nb_of_use_of_Joyau acrobatie/dégainage [3](0..100),
 'Joyau acrobatie/maître [3]': nb_of_use_of_Joyau acrobatie/maître [3](0..100),
 'Joyau aiguiseur/artisanat [3]': nb_of_use_of_Joyau aiguiseur/artisanat [3](0..100),
 'Joyau aiguiseur/attaque [3]': nb_of_use_of_Joyau aiguiseur/attaque [3](0..100),
 'Joyau aiguiseur/cogneur [3]': nb_of_use_of_Joyau aiguiseur/cogneur [3](0..100),
 'Joyau aiguiseur/dégainage [3]': nb_of_use_of_Joyau aiguiseur/dégainage [3](0..100),
 'Joyau aiguiseur/maître [3]': nb_of_use_of_Joyau aiguiseur/maître [3](0..100),
 'Joyau aiguiseur/paladin [3]': nb_of_use_of_Joyau aiguiseur/paladin [3](0..100),
 'Joyau armurier III [3]': nb_of_use_of_Joyau armurier III [3](0..100),
 'Joyau armurier/dracocide [3]': nb_of_use_of_Joyau armurier/dracocide [3](0..100),
 'Joyau armu

In [ ]:
weapon = (
    weapons
    #
    .filter(pl.col("name") == weapon_name)
    .explode("talents")
    .select(
        "name",
        *[
            pl.col("jewels").struct.field(str(i)).alias(f"jewel_lvl_{i}")
            for i in range(4)
        ],
        pl.col("talents").struct.field("name").alias("talent_name"),
        pl.col("talents").struct.field("lvl").alias("talent_lvl"),
    )
)
weapon
# weapon.schema

name,jewel_lvl_0,jewel_lvl_1,jewel_lvl_2,jewel_lvl_3,talent_name,talent_lvl
str,i64,i64,i64,i64,str,i64
"""Brisefoi Leibolkule""",0,1,1,1,"""Premier tir""",3
